In [27]:
import json
import pandas as pd
import pickle



json_record = {                       
    "id": 1,
    "username": "박준성",
    "email": "steepcastle@naver.com",
    "address": {
        "street": "율곡로",
        "suite": "Apt. 556",
        "city": "서울",
        "zipcode": "92998-3874"
    }
}

dfjs = pd.DataFrame(json_record)                 #dataframe 으로 변환
print(dfjs)

dftojs = dfjs.to_json(force_ascii=False)         #dataframe 을 json으로 변환해주는 매서드 to_json이다.to_확장자명 을 이용하여 csv json Excel 등 으로 객체를 변환한다.
                                                    #json 변환시 한글깨짐 현상이 일어난다. 아스키코드에는 한글이 없기때문이다. 아스키코드를 비활성화하여 깨짐을 막는다.
print(dftojs)



jsstr = json.dumps(json_record,ensure_ascii=False)    #json.dumps는 json자료형 위주로 json string 으로 변환할 수 있다. (df, dict, csv 등은 변환불가), 가독성을 올리고 싶으면 indent = N (n은정수)
                                     #json.dumps(데이터프레임) 같은 경우 is not JSON serializable이 아니라는 오류가 나오면서 변환이 안된다.
print(jsstr)

# with open('dfjs.json', 'w') as f:   #Object of type DataFrame is not JSON serializable 이라는 에러가 나온다. 데이터프레임에 들어있는 데이터가 json 모듈로서는 이해할 수 없는 구조로 되어있어
#     json.dump(dfjs, f, indent=2)     #어찌할수 없는 경우에 나타난다. dfjs와json_string 은 데이터프레임과 json 이라는 차이점만 가질 뿐 가지고 있는 내용은 같다. 그런데도 이런 에러가 나온다.
                                     #데이터의 양이 적을 경우엔 단순히 그 내용에 맞춰서 데이터에서 사용된 함수나 구조를 수정하면 될일이지만 너무나도 방대한 데이터에서는 일일히 수정이 불가능하므로
                                     #처리할 수 있는 데이터의 유형을 상황에 맞춰 변환해주는게 좋다. df->json or df->dict of dict -> json 등등

with open('dftojs.pickle', 'wb') as f:              #df 을 json 으로 변환한 후 pickle로 내보내는 함수다. 위의 함수들은 객체를 객체로 변환하지만 pickle은 객체를 메모리에서 기억장치로 저장한다.
    pickle.dump(dftojs, f, pickle.HIGHEST_PROTOCOL)

with open('dftojs.pickle', 'rb') as f:              #경로안에 있는 pickle 파일을 객체로 불러오는 함수다. 경로가 다르게 지정되어 있다면 변수를 선언하고 불러올 파일경로를 따로 지정해주어야 한다.
	data = pickle.load(f)



dfjs.to_pickle('dfjs.pkl')              #dataframe을 간단하게 pickle 로 내보내는 매서드다 

data = pd.read_pickle('dfjs.pkl')        #위와같이 간단하게 pickle파일을 열어주는 매서드

         id username                  email     address
city      1      박준성  steepcastle@naver.com          서울
street    1      박준성  steepcastle@naver.com         율곡로
suite     1      박준성  steepcastle@naver.com    Apt. 556
zipcode   1      박준성  steepcastle@naver.com  92998-3874
{"id":{"city":1,"street":1,"suite":1,"zipcode":1},"username":{"city":"박준성","street":"박준성","suite":"박준성","zipcode":"박준성"},"email":{"city":"steepcastle@naver.com","street":"steepcastle@naver.com","suite":"steepcastle@naver.com","zipcode":"steepcastle@naver.com"},"address":{"city":"서울","street":"율곡로","suite":"Apt. 556","zipcode":"92998-3874"}}
{"id": 1, "username": "박준성", "email": "steepcastle@naver.com", "address": {"street": "율곡로", "suite": "Apt. 556", "city": "서울", "zipcode": "92998-3874"}}


In [28]:
from fileinput import close
import pandas as pd
import os
import numpy
import json

path = os.getcwd() #실행폴더 확인
print(path)
file_name = 'df_summary_fullest.csv' #실행파일 확인
print(file_name)
file_path = os.path.join(path,file_name)  # directory, file 경로 합침 
print(file_path)

 
df = pd.read_csv(file_path, encoding='utf-8')        #불러올 파일의 경로, 폰트깨짐을 막기위해 파일별로 적정한 인코딩방식
dfhead = df.head(2)    #json모듈을 사용하는 dump, dumps 는 dataframe 타입을 json으로 변환하지 못한다.



def df_to_json_dump(dfhead) :                #json.dump의 경우 본래는 df구조를 json으로 변환하지 못한다. 따라서 df를 적절한 자료형으로 가공한후 다시 json으로 재가공 해야한다
  b = dfhead.to_dict()                        #이 사용자지정함수는 인자로 받은 객체(=dataframe)을 to_dict 매서드로 dict객체로 변환하고 이를 사용자가 입력한 json파일로 다시 변환하여 내보낸다.
  mkfile_name = input("파일명.json을 입력하시오.(ex : bald.json) : ")   #text 그대로가 아닌 json 바이너리로 내보내므로 한글이 깨지는 현상이 발생한다. 이는 json.dump 매서드가 내부의 자료형을 ascii 로 자동으로 변환하기 때문이며
                                              #이를 예방하고 싶다면 ensure_ascii = False 를 입력하며 아스키코드 인코딩을 False 로 해주면 된다.
  with open(mkfile_name, 'w') as f :
    json.dump(b, f, ensure_ascii = False)

#인풋내용
# 1) dftojson.json (dump)        2) jsontostring.json  (dumps)

df_to_json_dump(dfhead)       #df를 dict로 변환후 json 으로 직렬화 이후 내보냄


json_to_jsondumps = df_to_json_dump(dfhead)
json.dumps(json_to_jsondumps)         #json string 저장 파일명 입력, string 문자열은 한줄로 표현되기에 가시성이 안좋으므로 들여쓰기(=>indent=n [n은정수]) 추가하는게 좋다
 
#json.dumps는 현재 메모리에 띄워져 있는 json 자료형을 json string 으로 변환해주는 매서드다. 따라서 json객체로 자료를 변환해주는 선행작업이 필수다.
#TypeError: Object of type DataFrame is not JSON serializable 데이터프레임 객체는 json.dumps 매서드로는 json으로 직렬화가 불가능하다는 에러이다.





with open("dftojson.json", "r") as l:
  l_load = json.load(l)
  print(l_load)
#내보낸 json 바이너리를 읽기모드로 불러와 메모리에 파이썬객체로 띄운다.(=역직렬화)

json_loads = json.loads(json.dumps(json_to_jsondumps))
print(json_loads)
#json.loads 는 json string 을 파이썬객체로 역직렬화
#json.load 는 파일에서 json 형식 데이터를 읽고 파이썬객체로 역직렬화



    




c:\Users\zkshq\Documents\VSCodeProjects\jspsandbox
df_summary_fullest.csv
c:\Users\zkshq\Documents\VSCodeProjects\jspsandbox\df_summary_fullest.csv
{'Unnamed: 0': {'0': 0, '1': 1}, 'filename': {'0': 'C://case_한다_선고_결정_있다_없다_이다_판결_이유_latest_1017_1000000_2022-01-13-21-43-59\\1017_page\\case_1017_0.txt', '1': 'C://case_한다_선고_결정_있다_없다_이다_판결_이유_latest_1017_1000000_2022-01-13-21-43-59\\1017_page\\case_1017_1.txt'}, 'case_full_no': {'0': '대법원 2010. 1. 25.자 2009마2183 결정 ', '1': '대구고등법원 2010. 1. 22. 선고 2009누1533 판결 '}, 'acts': {'0': '【참조조문】lnfd 채무자 회생 및 파산에 관한 법률 제305조 제1항lnfd ', '1': '[1] 구 대구광역시세 감면 조례(2005. 9. 30. 대구광역시 조례 제3728호로 개정되기 전의 것) 제13조(현행 제19조 참조) '}, 'gists': {'0': nan, '1': '[1] 추징처분은 일단 감면요건에 해당하면 그 세액을 감면한 후에 당초의 감면취지에 합당한 사용을 하느냐에 대한 사후관리의 측면에서 규정한 것으로서 본래의 부과처분과는 그 요건을 달리하는 별개의 처분이므로, 추징처분이 해당 법에서 규정한 추징요건을 갖추지 못하였다면 그 추징처분은 위법한 처분이 되고, 감면요건을 갖추지 못하여 본래의 부과처분을 할 사유가 있다고 하더라도 그와 같은 사정이 위법한 추징처분을 적법한 것으로 보아야 할 사유가 될 수는 없다.lnfd '}, 'items': {'0': '【판시사항】lnfd 부채초과 상태인 개인 채무자가 파산

In [26]:
import pandas as pd

df_list = {"이름":["박주임","김종국","유재석","양세찬"],"수학":[10,20,30,40],"국어":[90,80,70,60,]}
df = pd.DataFrame(df_list, index = range(1,5))

print(df)


# dataframe은 시리즈의 집합이다. 따라서 행과 열의 처리시 시리즈를 머리속에 그리면서 코드를 작성하는게 좋다.
# pd.DataFrame매서드에 index = range(1,5)를 추가하여 행번호를 1부터5미만의 번호로 부여하였다.

df['과학'] = [11,22,33,44]
print(df)

df.insert(0, "주소",["서울", "부산", "경기", "강원"])
print(df)

# 객체.insert(삽입할 열번호, 열이름, 값목록)  으로도 시리즈를 추가할 수 있다.

df_list2 = {"성별":["남","남","남","남"],"혼인":["미혼","미혼","기혼","미혼"]}
df2 = pd.DataFrame(df_list2, index = range(1,5))
print(df2)

merge_df = pd.concat([df, df2],axis=1)
print(merge_df)

# 위처럼 두개 이상의 데이터프레임을 서로 합치는 것도 가능하다. 이때 axis=1을 지정해주어 행이아닌 열로 추가되도록 한다.(미지정시 서로가nan으로 채워진 두개의 DF생성)
# 만약 데이터프레임의 배열크기가 서로 맞지 않다면 오류가 나오므로 항상 shape를 맞추어야 한다.

set_df = df.set_index("이름")
print(set_df)

# 이미 존재하는 시리즈를 행이름으로 지정하는 매서드다.'이름'시리즈가 행이름으로 들어가고 나머지 시리즈와 한칸씩 당겨졌다.

reset_df = df.reset_index()
print(reset_df)

# 행이름에 index 정수로 이루어진 번호를 추가한다. 열을 행으로 바꾸거나 행record를 지정해준 dataframe에서 행번호를 추가할때 사용하는 것이다.
df_list3 = {"이름":["박사장","박차장","박대리"],"수학":[10,20,30],"국어":[90,80,70],"주소":["서울","강남","대구"]}
df3 = pd.DataFrame(df_list3)
df3.insert(0,"과학" , [12,23,34])
app_df = df.append(df3,ignore_index=True)

print(app_df)


# 기존에 있던 df 에 새로운 df3을 추가한다. ignore_index=True 는 새로추가되는 데이터 프레임의 행번호가 다시 0부터 시작하기 때문에 그에 따른 혼란을 없애고 
# 새로 만들어진 app_df 의 행번호를 만들어준다.
# 기존에 없던 df_list3 에 없는 "과학" 시리즈를 새로 생성하고 추가하여 준다. 
# append 매서드는 추후 삭제될 예정이므로 concat 매서드를 주로 사용해야 할것이다.


    이름  수학  국어
1  박주임  10  90
2  김종국  20  80
3  유재석  30  70
4  양세찬  40  60
    이름  수학  국어  과학
1  박주임  10  90  11
2  김종국  20  80  22
3  유재석  30  70  33
4  양세찬  40  60  44
   주소   이름  수학  국어  과학
1  서울  박주임  10  90  11
2  부산  김종국  20  80  22
3  경기  유재석  30  70  33
4  강원  양세찬  40  60  44
  성별  혼인
1  남  미혼
2  남  미혼
3  남  기혼
4  남  미혼
   주소   이름  수학  국어  과학 성별  혼인
1  서울  박주임  10  90  11  남  미혼
2  부산  김종국  20  80  22  남  미혼
3  경기  유재석  30  70  33  남  기혼
4  강원  양세찬  40  60  44  남  미혼
     주소  수학  국어  과학
이름                 
박주임  서울  10  90  11
김종국  부산  20  80  22
유재석  경기  30  70  33
양세찬  강원  40  60  44
   index  주소   이름  수학  국어  과학
0      1  서울  박주임  10  90  11
1      2  부산  김종국  20  80  22
2      3  경기  유재석  30  70  33
3      4  강원  양세찬  40  60  44
   주소   이름  수학  국어  과학
0  서울  박주임  10  90  11
1  부산  김종국  20  80  22
2  경기  유재석  30  70  33
3  강원  양세찬  40  60  44
4  서울  박사장  10  90  12
5  강남  박차장  20  80  23
6  대구  박대리  30  70  34


C:\Users\zkshq\AppData\Local\Temp\ipykernel_8476\208371354.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  app_df = df.append(df3,ignore_index=True)
